# Eigenvalue problem

### Initial tests:

Given the matrix A above:

In [47]:
import numpy as np
import math
A = np.array([[3, 4, 0],
              [4, 3, 0],
              [0, 0, 2]])

Solving the characteristic equation `det(A - λI) = 0` we get the following eigenvalues:

In [21]:
lambda1 = -1
lambda2 = 2
lambda3 = 7

Then, replacing those eigenvalues in `Ax = λx` we get the following normalized associated eigenvectors:

In [64]:
# associated with lambda1 = -1
x1 = np.array([0.7071,
              -0.7071,
               0.])


# associated with lambda2 = 2
x2 = np.array([0.2425,
              -0.9701,
               0.])

# associated with lambda3 = 7
x3 = np.array([0.7071,
               0.7071,
               0.])


Now that we have the exact solution, we'll use the QR method to find an approximate solution by leveraging the Householder transformation:

In [102]:
def multiply_by_scalar(matrix, scalar):
    
    
def dimension_of(matrix):
    return matrix.shape[0]

def nth_canonical_form(n, dimension):
    zeros = np.zeros(dimension)
    zeros[n-1] = 1
    return zeros

def norm(v):
    sum_square = 0
    for i in v:
        sum_square += math.pow(i, 2)
    return math.sqrt(sum_square)

def compute_sigma(a):
    return a[0] / math.fabs(a[])

a = A[0]
e1 = nth_canonical_form(3, dimension_of(A))
sigma = compute_sigma(a)
v1 = sigma*(norm(a)/norm(e1))

TypeError: only size-1 arrays can be converted to Python scalars

-1.0
